In [45]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/soniame/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml

import warnings
warnings.filterwarnings('ignore')

In [46]:
from src import utils
conn = utils.connect_athena(path='../configs/athena.yaml')

In [47]:
from tabulate import tabulate

In [51]:
def _extract_dictionary(database_name, table_name):
    df = pd.read_sql_query(f"""
        select * 
        from information_schema.columns
        where table_schema = '{database_name}' and table_name = '{table_name}';
        """, conn)
    
    return(df)

def _write_to_md(df, f):

    df['Description'] = ''
    
    print('\n', file=f)
    print('### ' + df.table_name[1] + '\n', file=f)
    print('Database: ' + df.table_schema[1] + '\n', file=f)
    print('Table: ' + df.table_name[1] + '\n', file=f)
    print(tabulate(df[['column_name', 'Description', 'data_type']].set_index('column_name'), 
                   tablefmt="pipe", headers="keys"), file=f)
    print('\n\n', file=f)
    
def write_dictionary(database_name, table_name, f):
    _write_to_md(_extract_dictionary(database_name, table_name), f)  
    

In [52]:
with open('../docs/Data Dictionary Internal.md', 'w') as f:
    write_dictionary(database_name = 'pwazetransformeddb', 
                     table_name = 'jams', f = f)
    write_dictionary(database_name = 'spd_sdv_waze_corona', 
                     table_name = 'prod_daily_daily', f = f)
